In [ ]:
import os
import sklearn
os.environ['CUDA_VISIBLE_DEVICES'] = "-1"
import warnings
warnings.filterwarnings('always')  # "error", "ignore", "always", "default", "module" or "once"

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 2
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
#cic2017 preprocses
!python3 lucid_dataset_parser.py --dataset_type DOS2017 --dataset_folder ./cic2017-training/ --packets_per_flow 100 --dataset_id DOS2017 --traffic_type all --time_window 100

In [ ]:
#preprocess
!python3 lucid_dataset_parser.py --preprocess_folder ./cic-processed-data/

In [ ]:
!python3 lucid_cnn.py --train ./cic-processed-data/  --epochs 50 --regularization l2 --dropout 0.5

In [ ]:
import tensorflow as tf

# Convert the model
lucid_model = tf.keras.models.load_model("final-model/100t-100n-IDS201X-LUCID.h5", compile=False)
converter = tf.lite.TFLiteConverter.from_keras_model(lucid_model) # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('100t-100n-IDS201X-LUCID.tflite', 'wb') as f:
  f.write(tflite_model)

In [26]:
# used in command line
tflite_convert \
  --keras_model_file=final-model/100t-100n-IDS201X-LUCID.h5 \
  --output_file=/final-model/100t-100n-IDS201X-LUCID.tflite


/home/zvd0712/miniconda3/envs/gpu/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


SyntaxError: cannot assign to operator (<ipython-input-26-5a85eb10a411>, line 1)

In [ ]:
#test cicddos
!python3 lucid_cnn.py --predict_live ./test-cicddos/SAT-01-12-2018_0500.pcap --model ./100t-100n-IDS201X-LUCID.tflite

In [ ]:
#test swat benign
!python3 lucid_cnn.py --predict_live ./test-swat/swat_benign.pcap --model ./100t-100n-IDS201X-LUCID.tflite

In [ ]:
#test swat ddos
!python3 lucid_cnn.py --predict_live ./test-swat/swat_attack.pcap --model ./100t-100n-IDS201X-LUCID.tflite

In [ ]:
#test cicddos benign
!python3 lucid_cnn.py --predict_live ./test-cicddos/cicddos_benign.pcap --model ./100t-100n-IDS201X-LUCID.tflite

In [ ]:
#test cicddos attack
!python3 lucid_cnn.py --predict_live ./test-cicddos/cicddos_attack.pcap --model ./100t-100n-IDS201X-LUCID.tflite